In [1]:
import pandas as pd

In [2]:
from google.colab import files
upload=files.upload()

Saving Dataset.xlsx to Dataset.xlsx


In [4]:
pip install scikeras[tensorflow]


In [5]:
pip install --upgrade tensorflow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 68.8 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.12.3
    Uninstalling tensorboard-2.12.3:
      Successfully uninstalled tensorboard-2.12.3
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.12.1
    Uninstalling tensorflow-2.12.1:
      Successfully

In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor

# Load the dataset from Excel
dataset_path = r"Dataset.xlsx"
df = pd.read_excel(dataset_path)

# Select features and target variable
features = df[['N', 'P', 'K', 'pH', 'rainfall', 'temperature', 'Area_in_hectares']]
target = df['Yield_ton_per_hec']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Random Forest Model for feature selection
rf_feature_selection_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_feature_selection_model.fit(X_train_scaled, y_train)

# Use feature importance from Random Forest for feature selection
feature_importance = rf_feature_selection_model.feature_importances_
selected_features_indices = feature_importance.argsort()[-5:][::-1]  # Select top 5 features
selected_features = X_train.columns[selected_features_indices]

# Use only the selected features
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

# Neural Network Model
from tensorflow.keras.regularizers import l2

# Neural Network Model with L2 Regularization
def create_nn_model_with_l2():
    model = Sequential()

    # Add a Dense layer with L2 regularization to the input layer
    model.add(Dense(64, input_dim=X_train_selected.shape[1], activation='relu', kernel_regularizer=l2(0.01)))

    # Add more Dense layers with L2 regularization
    model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))

    # Output layer with linear activation (for regression tasks)
    model.add(Dense(1, activation='linear', kernel_regularizer=l2(0.01)))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    return model



# Wrap the Keras model for scikit-learn compatibility
keras_regressor = KerasRegressor(build_fn=create_nn_model, epochs=50, batch_size=32, verbose=0)

# Gradient Boosting Model for prediction
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb_model.fit(X_train_selected, y_train)

# Create a stacked model with Random Forest, Neural Network, and Gradient Boosting
stacked_model = StackingRegressor(
    estimators=[
        ('rf', rf_feature_selection_model),
        ('nn', keras_regressor),
        ('gb', gb_model)
    ],
    final_estimator=RidgeCV()
)

# Train the stacked model
stacked_model.fit(X_train_selected, y_train)

# Predictions using the stacked model
stacked_predictions = stacked_model.predict(X_test_selected)

# Evaluate the stacked model
stacked_mse = mean_squared_error(y_test, stacked_predictions)
stacked_r2 = r2_score(y_test, stacked_predictions)

print("Stacked Model (Random Forest for feature selection + Neural Network + Gradient Boosting):")
print(f'Mean Squared Error: {stacked_mse}')
print(f'R-squared: {stacked_r2}')


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Stacked Model (Random Forest for feature selection + Neural Network + Gradient Boosting):
Mean Squared Error: 348.33467492110816
R-squared: 0.07918657078492297


In [8]:
import numpy as np

In [11]:
epsilon = 1e-10  # Small constant to avoid division by zero

# Calculate Mean Absolute Percentage Error (MAPE)
mape = np.mean(np.abs((y_test - stacked_predictions) / (y_test + epsilon))) * 100

# Calculate the percentage of predictions within a certain error range (e.g., 10%)
within_10_percent = np.mean(np.abs((y_test - stacked_predictions) / (y_test + epsilon)) <= 0.10) * 100

print(f'Mean Absolute Percentage Error (MAPE): {mape:.2f}%')
print(f'Percentage of predictions within 10% error: {within_10_percent:.2f}%')


Mean Absolute Percentage Error (MAPE): 123731965345.65%
Percentage of predictions within 10% error: 11.96%


In [12]:
# Define a threshold for acceptable predictions (e.g., 10% error)
threshold = 0.10

# Convert regression predictions to binary classification
binary_predictions = np.abs((y_test - stacked_predictions) / y_test) <= threshold

# Convert true values to binary based on the same threshold
binary_true_values = np.abs((y_test - y_test.mean()) / y_test) <= threshold

# Calculate precision, recall, and F1-score
precision = np.sum(binary_predictions & binary_true_values) / np.sum(binary_predictions)
recall = np.sum(binary_predictions & binary_true_values) / np.sum(binary_true_values)
f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-score: {f1_score:.2f}')


Precision: 0.02
Recall: 0.09
F1-score: 0.03


In [13]:
# R-squared value for regression accuracy
print(f'R-squared: {stacked_r2:.4f}')

# Alternatively, convert it to a percentage
r2_percentage = stacked_r2 * 100
print(f'R-squared Percentage: {r2_percentage:.2f}%')


R-squared: 0.0884
R-squared Percentage: 8.84%


In [14]:
# Calculate the percentage of predictions within a certain range
percentage_threshold = 10  # Set the percentage threshold for correctness

# Calculate the absolute percentage difference between true and predicted values
percentage_difference = abs((stacked_predictions - y_test) / y_test) * 100

# Count the number of predictions within the specified threshold
correct_predictions_count = sum(percentage_difference <= percentage_threshold)

# Calculate the percentage of correct predictions
percentage_correct_predictions = (correct_predictions_count / len(y_test)) * 100

print("Stacked Model (Random Forest for feature selection + Neural Network + Gradient Boosting):")
print(f'Mean Squared Error: {stacked_mse}')
print(f'R-squared: {stacked_r2}')
print(f'Percentage of Predictions within {percentage_threshold}%: {percentage_correct_predictions}%')


Stacked Model (Random Forest for feature selection + Neural Network + Gradient Boosting):
Mean Squared Error: 344.8674235910564
R-squared: 0.08835215726550671
Percentage of Predictions within 10%: 11.957936905358038%


In [16]:
(1-stacked_mse/(target.max()-target.min()))*100

96.48130370787618

In [18]:
rmse = np.sqrt(stacked_mse)
rmse_percentage = (1 - rmse / (target.max() - target.min())) * 100
rmse_percentage


99.81052335025893

In [20]:
tolerance = 0.1  # Adjust the tolerance as needed
correct_predictions = np.sum(np.abs(stacked_predictions - y_test) < tolerance)
accuracy_percentage = (correct_predictions / len(y_test)) * 100
accuracy_percentage

8.943415122684026

In [22]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, stacked_predictions)
mae_percentage = (1 - mae / target.mean()) * 100
mae_percentage


44.49202270422747

In [23]:
from sklearn.metrics import mean_absolute_error, explained_variance_score, r2_score

# ... (Your existing code for training the stacked model)

# Predictions on the training set
train_stacked_predictions = stacked_model.predict(X_train_selected)

# Metrics for the training set
train_stacked_mse = mean_squared_error(y_train, train_stacked_predictions)
train_r2 = r2_score(y_train, train_stacked_predictions)
train_mae = mean_absolute_error(y_train, train_stacked_predictions)

# Print metrics for the training set
print("Training Set Metrics:")
print(f'Training Mean Squared Error: {train_stacked_mse}')
print(f'Training R-squared: {train_r2}')
print(f'Training Mean Absolute Error: {train_mae}')

# Predictions on the testing set
stacked_predictions = stacked_model.predict(X_test_selected)

# Metrics for the testing set
stacked_mse = mean_squared_error(y_test, stacked_predictions)
stacked_r2 = r2_score(y_test, stacked_predictions)
stacked_mae = mean_absolute_error(y_test, stacked_predictions)

# Print metrics for the testing set
print("\nTesting Set Metrics:")
print(f'Testing Mean Squared Error: {stacked_mse}')
print(f'Testing R-squared: {stacked_r2}')
print(f'Testing Mean Absolute Error: {stacked_mae}')


Training Set Metrics:
Training Mean Squared Error: 1289.4350988409574
Training R-squared: 0.037429953825287465
Training Mean Absolute Error: 2.1348309376470054

Testing Set Metrics:
Testing Mean Squared Error: 344.8674235910564
Testing R-squared: 0.08835215726550671
Testing Mean Absolute Error: 2.1821015100255905


In [ ]:
2+3